# This is v3 of the recommender system.

To improve on v2, we add distributed computing support with Spark and Petastorm to ingest training data stored in Amazon S3 at scale.

In [ ]:
!pip install tqdm
!pip install sentence-transformers
!pip install annoy
!pip install torchsummary
!pip install pyspark
!pip install petastorm

In [20]:
import os
import torch
import torch.nn as nn
import torchvision.models as models
from PIL import Image
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
import numpy as np
from tqdm import tqdm
from annoy import AnnoyIndex
from sklearn.model_selection import train_test_split
from torchsummary import summary

In [18]:
from pyspark.sql import SparkSession, functions as F
from pyspark import SparkConf, SparkContext

# create spark session
spark = SparkSession.builder.getOrCreate()
spark.version
sc = spark.sparkContext

25/02/03 21:04:12 WARN Utils: Your hostname, Tylers-MacBook-Pro-5.local resolves to a loopback address: 127.0.0.1; using 192.168.10.138 instead (on interface en0)
25/02/03 21:04:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/03 21:04:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [21]:
accessKeyId = os.getenv("AWS_ACCESS_KEY_ID")
secretAccessKey = os.getenv("AWS_SECRET_ACCESS_KEY")

sc._jsc.hadoopConfiguration().set('fs.s3a.access.key', accessKeyId)
sc._jsc.hadoopConfiguration().set('fs.s3a.secret.key', secretAccessKey)
sc._jsc.hadoopConfiguration().set('fs.s3a.path.style.access', 'true')
sc._jsc.hadoopConfiguration().set('fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem')
 
sc._jsc.hadoopConfiguration().set('fs.s3a.endpoint', 's3.amazonaws.com')

In [ ]:
!pkill -f pyspark

In [ ]:
# create a metadata file for fetching images from s3 and store in paraquet format on s3
# metadata contains s3_path, + everything in styles.csv
df = spark.read.csv("s3a://closetspace/train_data/metadata/styles.csv", header=True, inferSchema=True)
# add the s3_path to the df
s3_image_prefix = "s3a://closetspace/train_data/images/"
df = df.withColumn('s3_path', F.concat(F.lit(s3_image_prefix), df.id, F.lit('.jpg')))
df.show()

# write the full metadata df to s3a://closetspace/train_data/metadata/full_metadata.paraquet
df.write.mode("overwrite").parquet("s3a://closetspace/train_data/metadata/full_metadata.paraquet")

In [ ]:
df_testing_read = spark.read.parquet("s3a://closetspace/train_data/metadata/full_metadata.paraquet")
df_testing_read.show()

In [ ]:
# create train, val, and test splits, saving each as their own paraquet file on s3
# Split the data: 80% train, 10% validation, 10% test
seed = 21
train_df, temp_df = df.randomSplit([0.8, 0.2], seed=seed)
val_df, test_df = temp_df.randomSplit([0.5, 0.5], seed=seed)

train_df.write.mode("overwrite").parquet("s3a://closetspace/train_data/metadata/train.paraquet")
val_df.write.mode("overwrite").parquet("s3a://closetspace/train_data/metadata/val.paraquet")
test_df.write.mode("overwrite").parquet("s3a://closetspace/train_data/metadata/test.paraquet")

In [ ]:
# Define constants
def convert_to_rgb(image):
    # Convert RGBA or grayscale to RGB
    if image.mode != "RGB":
        image = image.convert("RGB")
    return image

transform = transforms.Compose([
    transforms.Lambda(convert_to_rgb),
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
batch_size=32
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(device)

In [ ]:
!pip install -U pyarrow petastorm
!pip install -U s3fs

In [ ]:
# create the dataloader using Petastorm (this seems deprecated)
import petastorm
from petastorm import make_reader
from petastorm.pytorch import DataLoader # 2 imports to dataloader?
import boto3
import io
from PIL import Image
import torch
from torchvision import transforms

def load_data(sample):
    s3_client = boto3.client("s3")
    bucket, key = sample["s3_path"].replace("s3a://", "").split("/", 1)

    # read image from S3
    response = s3_client.get_object(Bucket=bucket, Key=key)
    image = Image.open(io.BytesIO(response["Body"].read())).convert("RGB")

    # apply transform for ResNet50
    image_tensor = transform(image)

    description = sample["productDisplayName"]

    return image_tensor, description

train_reader = make_reader("s3a://closetspace/train_data/metadata/train.parquet", reader_pool_type="thread", shuffle=True)
val_reader = make_reader("s3a://closetspace/train_data/metadata/val.parquet", reader_pool_type="thread")
test_reader = make_reader("s3a://closetspace/train_data/metadata/test.parquet", reader_pool_type="thread")

train_dataloader = DataLoader(train_reader, batch_size=32, transform=load_data)
val_dataloader = DataLoader(val_reader, batch_size=32, transform=load_data)
test_dataloader = DataLoader(test_reader, batch_size=32, transform=load_data)

# then, train as usual! 

In [ ]:
!pip install boto3
!pip install pyarrow

In [ ]:
# replace above cell with aws s3 connector
import boto3
import io
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms
import pandas as pd

# S3 client
s3_client = boto3.client('s3')
response = s3_client.list_buckets()

print(response['Buckets'])

bucket_name="closetspace"
response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix="train_data/metadata")

for obj in response.get('Contents', []):
    print(obj['Key'])


In [39]:
# Custom Dataset
# the commented out code is the implementation for no spark
class S3ImageDataset(Dataset):
    def __init__(self, parquet_path, bucket_name, transform=None):
        self.bucket_name = bucket_name
        self.parquet_path = parquet_path
        self.transform = transform
        self.metadata_df = self.load_metadata_from_s3(parquet_path)

    # def load_metadata_from_s3(self, s3_path):
    #     bucket_name, prefix = s3_path.replace("s3a://", "").split("/", 1)
        
    #     # list all the parquet files in the S3 directory
    #     response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
    #     files = [content['Key'] for content in response.get('Contents', []) if content['Key'].endswith('.parquet')]
        
    #     # read all the paraquet files in Paraquet folder
    #     dfs = []
    #     for file_key in files:
    #         response = s3_client.get_object(Bucket=bucket_name, Key=file_key)
    #         parquet_data = response['Body'].read()
    #         df = pd.read_parquet(io.BytesIO(parquet_data))
    #         dfs.append(df)

    #     # Concatenate all DataFrames (all part files) into one DataFrame
    #     full_metadata_df = pd.concat(dfs, ignore_index=True)
    #     return full_metadata_df

    def load_metadata_from_s3(self, s3_path):
        bucket_name, prefix = s3_path.replace("s3a://", "").split("/", 1)
        response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
        
        # List all Parquet files in the S3 directory
        files = [content['Key'] for content in response.get('Contents', []) if content['Key'].endswith('.parquet')]
        
        # Use Spark to read all parquet files into a single DataFrame
        if files:
            s3_paths = [f"s3a://{bucket_name}/{file_key}" for file_key in files]
            
            # Read all Parquet files into a single Spark DataFrame
            metadata_df = spark.read.parquet(*s3_paths)
        else:
            metadata_df = spark.createDataFrame([], schema=None)

        metadata_df = metadata_df.withColumn("index_column", F.monotonically_increasing_id())
        return metadata_df

    # def __len__(self):
    #     return len(self.metadata_df)
    def __len__(self):
        return self.metadata_df.count()

    # def __getitem__(self, idx):
    #     s3_path = self.metadata_df.iloc[idx]['s3_path']
    #     description = self.metadata_df.iloc[idx]['productDisplayName']

    #     bucket_name, file_key = s3_path.replace("s3a://", "").split("/", 1)
        
    #     # read the image from S3
    #     response = s3_client.get_object(Bucket=self.bucket_name, Key=file_key)
    #     image = Image.open(io.BytesIO(response['Body'].read())).convert("RGB")
        
    #     if self.transform:
    #         image = self.transform(image)

    #     return image, description
    def __getitem__(self, idx):
        row = self.metadata_df.filter(F.col('index_column') == idx).collect()

        s3_path = row[0]['s3_path']
        description = row[0]['productDisplayName']
        
        bucket_name, file_key = s3_path.replace("s3a://", "").split("/", 1)

        response = s3_client.get_object(Bucket=bucket_name, Key=file_key)
        image_data = response['Body'].read()

        image = Image.open(io.BytesIO(image_data))

        if self.transform:
            image = self.transform(image)

        return image, description

bucket_name = "closetspace"

train_path = "s3a://closetspace/train_data/metadata/train.paraquet"
val_path = "s3a://closetspace/train_data/metadata/val.paraquet"
test_path = "s3a://closetspace/train_data/metadata/test.paraquet"

train_dataset = S3ImageDataset(parquet_path, bucket_name, transform)
val_dataset = S3ImageDataset(parquet_path, bucket_name, transform)
test_dataset = S3ImageDataset(parquet_path, bucket_name, transform)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [40]:
for images, descriptions in test_dataloader:
    print(images.shape)
    print(descriptions)
    break

torch.Size([32, 3, 224, 224])
('Turtle Check Men Navy Blue Shirt', 'Peter England Men Party Blue Jeans', 'Titan Women Silver Watch', 'Manchester United Men Solid Black Track Pants', 'Puma Men Grey T-shirt', 'Inkfruit Mens Chain Reaction T-shirt', 'Fabindia Men Striped Green Shirt', 'Jealous 21 Women Purple Shirt', 'Puma Men Pack of 3 Socks', 'Skagen Men Black Watch', 'Puma Men Future Cat Remix SF Black Casual Shoes', 'Fossil Women Black Huarache Weave Belt', 'Fila Men Cush Flex Black Slippers', 'Murcia Women Blue Handbag', 'Ben 10 Boys Navy Blue Slippers', 'Reid & Taylor Men Check Purple Shirts', 'Police Men Black Dial Watch PL12889JVSB', 'Gini and Jony Girls Knit White Top', 'Bwitch Beige Full-Coverage Bra BW335', 'Baggit Women Brown Handbag', 'CASIO G-Shock Men Black Digital Watch G-7710-1DR G223', 'ADIDAS Men Spry M Black Sandals', 'Timberland Unisex Rubber Sole Brush Shoe Accessories', 'ADIDAS Men Lfc Auth Hood Grey Sweatshirts', 'David Beckham Signature Men Deos', 'Buckaroo Men Fl

In [ ]:
# Define a custom dataset
class ImageDataset(Dataset):
    def __init__(self, image_paths, transform):
        self.image_paths = image_paths
        self.transform = transform
    def __len__(self):
        return len(self.image_paths)
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path)
        return self.transform(image), img_path # return (image, path)

In [ ]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        resnet50 = models.resnet50(pretrained=True)
        self.encoder = nn.Sequential(*(list(resnet50.children())[:-1])) # remove fc layer used for classification

        # freeze layers up to 3 to retain information learned from pretrained weights
        for name, layer in self.encoder.named_children():
            if name in['0', '1', '2', '3']:
                for param in layer.parameters():
                    param.requires_grad = False
            
    def forward(self, x):
        latent = self.encoder(x).view(x.size(0), -1)
        return latent

class Decoder(nn.Module):
    def __init__(self, latent_dim=2048):
        super(Decoder, self).__init__()
        self.decoder = nn.Sequential(
            # Fully connected layer to expand the latent vector
            nn.Linear(latent_dim, 8 * 8 * 256),  # 8x8 spatial dimension and 256 channels
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(8 * 8 * 256),
            
            # Reshape to (B, 256, 8, 8) via view
            nn.Unflatten(1, (256, 8, 8)),
            
            # Upsampling layers (transpose convolutions)
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=2),    # 8x8 -> 14x14
            nn.ReLU(inplace=True),
            
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),     # 14x14 -> 28x28
            nn.ReLU(inplace=True),
            
            nn.ConvTranspose2d(64, 32, kernel_size=4, stride=2, padding=1),      # 28x28 -> 56x56
            nn.ReLU(inplace=True),
            
            nn.ConvTranspose2d(32, 16, kernel_size=4, stride=2, padding=1),      # 56x56 -> 112x112
            nn.ReLU(inplace=True),
            
            nn.ConvTranspose2d(16, 3, kernel_size=4, stride=2, padding=1),       # 112x112 -> 224x224
            nn.Sigmoid()  # Scaling the output to [0, 1] for RGB images
        )
    
    def forward(self, x):
        return self.decoder(x)

class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = Encoder()
        self.decoder = Decoder(latent_dim=2048)

    def forward(self, x):
        latent = self.encoder(x)
        reconstructed = self.decoder(latent)
        return latent, reconstructed

In [ ]:
decoder = Autoencoder().decoder
summary(decoder, (2048,))

In [ ]:
model = Autoencoder().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)

# Train, val, test splits (60, 20, 20)
image_folder = "./fashion-dataset/images"
image_paths = [os.path.join(image_folder, fname) for fname in os.listdir(image_folder)]

train_val_paths, test_paths = train_test_split(image_paths, test_size=0.2)
train_paths, val_paths = train_test_split(train_val_paths, test_size=0.25)

train_dataset = ImageDataset(train_paths, transform)
val_dataset = ImageDataset(val_paths, transform)
test_dataset = ImageDataset(test_paths, transform)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# Training loop
def train_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    # Define the mean and std for un-normalization (match your transform normalization)
    mean = torch.tensor([0.485, 0.456, 0.406], device=device).view(1, 3, 1, 1).to(device)
    std = torch.tensor([0.229, 0.224, 0.225], device=device).view(1, 3, 1, 1).to(device)
    
    for images, paths in tqdm(dataloader, desc="Training loop", unit='batch', leave=True):
        images = images.to(device)
        optimizer.zero_grad()
        latent, reconstructed = model(images)

        # Un-normalize the input images
        unnormalized_images = images * std + mean
        
        loss = criterion(reconstructed, unnormalized_images)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(dataloader)

epochs = 30
for epoch in range(epochs):
    train_loss = train_epoch(model, train_dataloader, criterion, optimizer, device)
    print(f"Epoch {epoch+1}/{epochs}, Loss: {train_loss:.4f}")

In [ ]:
torch.save(model.state_dict(), "resnet50_autoencoder.pth")

In [ ]:
model = Autoencoder().to(device)
model.load_state_dict(torch.load("resnet50_autoencoder.pth", weights_only=True))

In [ ]:
def evaluate_model_loss(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    # Define the mean and std for un-normalization (match your transform normalization)
    mean = torch.tensor([0.485, 0.456, 0.406], device=device).view(1, 3, 1, 1).to(device)
    std = torch.tensor([0.229, 0.224, 0.225], device=device).view(1, 3, 1, 1).to(device)
    for images, paths in tqdm(dataloader, desc="Evaluation loop", unit='batch', leave=True):
        images = images.to(device)
        latent, reconstructed = model(images)

        # Un-normalize the input images
        unnormalized_images = images * std + mean
        
        loss = criterion(reconstructed, unnormalized_images)
        running_loss += loss.item()
    return running_loss / len(dataloader)

val_loss = evaluate_model_loss(model, val_dataloader, criterion, device)
print(val_loss)

In [ ]:
def embed_image_dataset(dataloader, model, device, save_to_file=False, filename=""):
    latent_representations = {}
    model.to(device)
    model.eval()
    with torch.no_grad():
        for images, paths in tqdm(dataloader, desc="Processing Images", unit='batch'):
            images = images.to(device) # Output: [batch_size, 3, 224, 224]
            features = model(images)[0] # Output: [batch_size, 2048]
            if features.size(0) > 1:
                features = features.squeeze()
            for path, feature in zip(paths, features.cpu()):
                latent_representations[path] = feature.numpy()
    if save_to_file:
        np.save(filename, latent_representations)

    return latent_representations

In [ ]:
# load the wardrobe dataset and compute embeddings
wardrobe_folder = "./sample-wardrobe/images"
wardrobe_paths = [os.path.join(wardrobe_folder, fname) for fname in os.listdir(wardrobe_folder)]
wardrobe_dataset = ImageDataset(wardrobe_paths, transform)
wardrobe_dataloader = DataLoader(wardrobe_dataset, batch_size=batch_size, shuffle=False)
wardrobe_lat_rep = embed_image_dataset(wardrobe_dataloader, model, device)

In [ ]:
inventory_folder = "./nordstrom-data/images"
inventory_paths = [os.path.join(inventory_folder, fname) for fname in os.listdir(inventory_folder)]
inventory_dataset = ImageDataset(inventory_paths, transform)
inventory_dataloader = DataLoader(inventory_dataset, batch_size=batch_size, shuffle=False)
inventory_lat_rep = embed_image_dataset(inventory_dataloader, model, device, True, "lat_rep_inventory_ft.npy")

In [ ]:
# create list of fashion dataset embeddings and paths
inventory_lat_rep = np.load("lat_rep_inventory_ft.npy", allow_pickle=True).item()
inventory_img_paths = list(inventory_lat_rep.keys())
inventory_features = np.array(list(inventory_lat_rep.values()))

# create list of wardrobe embeddings and paths
wardrobe_paths = list(wardrobe_lat_rep.keys())
wardrobe_features = np.array(list(wardrobe_lat_rep.values()))

# get the mean embedding of all items in wardrobe
mean_embedding = np.mean(wardrobe_features, axis=0)

In [ ]:
# Perform Annoy
embedding_dim = 2048  # Original dimensionality
annoy_index = AnnoyIndex(embedding_dim, metric='euclidean')

# Add all items to Annoy index
for i, embedding in enumerate(inventory_features):
    annoy_index.add_item(i, embedding)

# Build the index
n_trees = 50
annoy_index.build(n_trees)  # Number of trees

# Query the index
n_neighbors = 10
indices = annoy_index.get_nns_by_vector(mean_embedding, n_neighbors, include_distances=True)

print("Recommended indices:", indices[0])
for idx in indices[0]:
    # print(fd_img_paths[idx])
    im = Image.open(inventory_img_paths[idx])
    im.show()

In [ ]:
# Encoder for metadata

In [ ]:
from sentence_transformers import SentenceTransformer
import json
import csv

In [ ]:
model = SentenceTransformer("all-MiniLM-L6-v2")
# example input: 
# metadata = ["Nike", "Nike Pegasus 40 White/Black"] # dim = d
# embedding = model.encode(metadata) # shape = [d x 384]

In [ ]:
inventory_metadata_rep = {}
formatted_products = {}
with open('./nordstrom-data/nordstrom_data.json', 'r') as f:
    formatted_products = json.load(f)

for path in tqdm(inventory_img_paths):
    base_path = os.path.basename(path)
    product = formatted_products[base_path]
    metadata = [product['brand'], product['name']]
    inventory_metadata_rep[path] = np.array(model.encode(metadata)).flatten()

np.save("lat_rep_inventory_metadata.npy", inventory_metadata_rep)

In [ ]:
wardrobe_metadata_rep = {}
wardrobe_products = {}
with open("./sample-wardrobe/metadata.csv", mode="r") as file:
    csv_reader = csv.DictReader(file)  # DictReader reads rows as dictionaries
    for row in csv_reader:
        src = row['filename']
        wardrobe_products[src] = row

for path in wardrobe_paths:
    base_path = os.path.basename(path)
    product = wardrobe_products[base_path]
    metadata = [product['brand'], product['name']]
    wardrobe_metadata_rep[path] = np.array(model.encode(metadata)).flatten()

In [ ]:
# concat embeddings
inventory_concat_embedding = {}
wardrobe_concat_embedding = {}

for path in inventory_img_paths:
    inventory_concat_embedding[path] = np.concatenate((inventory_lat_rep[path], inventory_metadata_rep[path]))

for path in wardrobe_paths:
    wardrobe_concat_embedding[path] = np.concatenate((wardrobe_lat_rep[path], wardrobe_metadata_rep[path]))

In [ ]:
# OPTIONAL: save concatenated embeddings to file
np.save("inventory_concat_embed_v2.npy", inventory_concat_embedding)
np.save("wardrobe_concat_embed_v2.npy", wardrobe_concat_embedding)

In [ ]:
inventory_concat_embedding = np.load("inventory_concat_embed_v2.npy", allow_pickle=True).item()
wardrobe_concat_embedding = np.load("wardrobe_concat_embed_v2.npy", allow_pickle=True).item()

In [ ]:
inventory_img_paths = list(inventory_concat_embedding.keys())
inventory_features = np.array(list(inventory_concat_embedding.values()))

# create list of wardrobe embeddings and paths
wardrobe_paths = list(wardrobe_concat_embedding.keys())
wardrobe_features = np.array(list(wardrobe_concat_embedding.values()))

# grab the mean embedding
mean_embedding = np.mean(wardrobe_features, axis=0)

In [ ]:
# Perform Annoy
embedding_dim = 2816  # Original dimensionality
annoy_index = AnnoyIndex(embedding_dim, metric='euclidean')

# Add all items to Annoy index
for i, embedding in enumerate(inventory_features):
    annoy_index.add_item(i, embedding)

# Build the index
n_trees = 50
annoy_index.build(n_trees)  # Number of trees

# Query the index
n_neighbors = 10
indices = annoy_index.get_nns_by_vector(mean_embedding, n_neighbors, include_distances=True)

for idx in indices[0]:
    im = Image.open(inventory_img_paths[idx])
    im.show()